# Import

In [ ]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os

# Load modules

In [ ]:
try: 
    pulsedmaster
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
try: 
    scanner
except NameError:
    manager.startModule('logic', 'scannerlogic')
try: 
    optimizer
except NameError:
    manager.startModule('logic', 'optimizerlogic')
    
try:
    sequencegenerator
except NameError:
    manager.startModule('logic', 'sequencegeneratorlogic')
    
sequencegenerator = sequencegeneratorlogic
pulsedmaster = pulsedmasterlogic
pulsedmeasurement = pulsedmeasurementlogic

# Method definitions

### Utility methods:

In [ ]:
def write_to_logfile(nametag, timestamp,name, **kwargs):
    """ Write parameters to custom logfile with name nametag """
    if type(timestamp) is not str:
        timestamp = str(timestamp)
    parameters = list(kwargs)
    if len(parameters) == 1 and type(kwargs[parameters[0]]) is OrderedDict:
        param_dict = kwargs[parameters[0]]
        parameters = list(param_dict)
        kwargs = param_dict
    log_dir = pulsedmeasurement._save_logic.get_path_for_module('CustomLogfiles')
    log_path = os.path.join(log_dir, nametag + '.txt')
    if not os.path.isfile(log_path):
        with open(log_path, 'w') as logfile:
            logfile.write('# timestamp\t')
            logfile.write('# Name\t')
            for param in parameters:
                logfile.write(param + '\t')
            logfile.write('\n#\n')
    with open(log_path, 'a') as logfile:
        logfile.write(timestamp + '\t')
        logfile.write(name + '\t')
        for param in parameters:
            logfile.write('{0:3.6e}\t'.format(kwargs[param]))
        logfile.write('\n')
    return

# Input parameters

In [ ]:
# static hardware parameters:
setup = OrderedDict()
setup['sampling_freq'] = pulsedmeasurementlogic.sample_rate
setup['fc_binwidth'] = pulsedmeasurementlogic.fast_counter_binwidth
setup['wait_length'] = 1.5e-6
setup['aom_delay'] = 500e-9
#setup['channel_config_name'] = pulsedmeasurement.current_channel_config_name
setup['seq_trig']=''
setup['gate']='d_ch2'
setup['NV_name']='NV1'
setup['poi']='poi_20170908_1920_43_092302'
setup['laser_length'] = 3.0e-6
setup["min_counts"] = 40000 # if after a refocus only 40kcounts are measured the refocus is repeated up to max 3 times
# Set amplitude in logic and pulser hardware:
amp_dict = OrderedDict()
amp_dict['a_ch1'] = 0.25
amp_dict['a_ch2'] = 0.0
AWG5014C.amplitude_dict = amp_dict
#pulsedmaster.pulse_generator_settings_changed(setup['sampling_freq'], setup['channel_config_name'], amp_dict, False)

# measurement start input parameters

# for Rabi:
rabi = OrderedDict()
rabi['mw_amp'] = 0.25
rabi['mw_freq'] = 100.0e6 
rabi['tau_start'] = 300e-9
rabi['tau_step'] = 10e-9
rabi['number_of_taus'] = 60
rabi['measurement_time'] = 30


# for single frequency XY8-N:
xy8 = OrderedDict()
xy8['mw_amp'] = 1.0
xy8['mw_freq'] = 3980.34e6
xy8['rabi_period'] = 32.743942*1e-9
xy8['order'] = 4
xy8['tau_start'] = 5154e-9
xy8['tau_incr'] = 210e-9
xy8['points'] = 8
xy8['measurement_time'] = 7200
xy8['refocus_interval'] = 600
xy8['alternating']=True


# for pulsed ODMR:
odmr = OrderedDict()
odmr['mw_amp'] = 0.05
odmr['pi_length'] = 1./7.164e6/2.
odmr['resonance_freq'] = 2002.5e6+2.15e6
odmr['freq_res'] = 1e6
odmr['points'] = 100
odmr['measurement_time'] = 30
#odmr['fit_function']='15N'
odmr['fit_function']='Lorentian'
#odmr['fit_function']='14N'

### Measurement methods:

In [ ]:
def refocus_position():
    # switch on laser
    pulsedmaster.load_asset_into_channels('Laser_On')
    while pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.5)
    pulsedmaster.toggle_pulse_generator(True)
    # perform refocus
    scanner.stop_scanning()
    crosshair_pos = scanner.get_position()
    optimizer.start_refocus(initial_pos=crosshair_pos)
    while optimizer.getState() == 'idle':
        time.sleep(0.2)
    while optimizer.getState() != 'idle':
        time.sleep(0.2)
    scanner.set_position('optimizer', x=optimizer.optim_pos_x, y=optimizer.optim_pos_y, z=optimizer.optim_pos_z, a=0.0)
    time.sleep(1)
    # switch off laser
    pulsedmaster.toggle_pulse_generator(False)
    
    # write new position to log file return
    timestamp = str(datetime.datetime.now())
    new_x = optimizer.optim_pos_x
    new_y = optimizer.optim_pos_y
    new_z = optimizer.optim_pos_z
#    drift = np.sqrt((new_x-position['x'])**2 + (new_y-position['y'])**2 + (new_z-position['z'])**2)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=new_x, y_pos=new_y, z_pos=new_z)
    return new_x, new_y, new_z

def refocus_poi(poi=None, max_drift=0.3,min_counts=setup["min_counts"]):
    if poi==None:
        return False
    pulsedmaster.load_asset_into_channels('Laser_On')
    while pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.5)
    pulsedmaster.toggle_pulse_generator(True)
    # perform refocus
    scanner.stop_scanning()
    poimanager.go_to_poi(poi)
    pos_start = scanner.get_position()
    setup['NV_name']=poimanager.poi_list[poi]._name
    poimanager.optimise_poi(poi)
    while optimizer.getState() == 'idle':
        time.sleep(0.2)
    while optimizer.getState() != 'idle':
        time.sleep(0.2)
    time.sleep(5)
    pos_end=scanner.get_position()
    print(counter.countdata.mean(),abs(pos_end[0]-pos_start[0]))
    if (abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counter.countdata.mean()<min_counts):
        scanner.set_position('',x=pos_start[0],y=pos_start[1],z=pos_start[2])
        poimanager.set_new_position(poi,pos_start)
        print(abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counter.countdata.mean()<min_counts)
        return False
        write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_start[0], y_pos=pos_start[1], z_pos=pos_start[2],sucess=0)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_end[0], y_pos=pos_end[1], z_pos=pos_end[2],sucess=1)
    return True

def do_rabi(generate_new=True, save_tag=None):
    
    length = rabi['number_of_taus'] * rabi['tau_step']
            
    # sanity check for long sequences:
    if length > 3e-3:
        print("Rabi exceeds 3 ms too long for AWG5014C")
        return True,0,0,0,0
    # generate ensemble object
#     rabi['tau_res'] = rabi['length'] / (rabi['points'] - 1)
    tau_arr = np.arange(rabi['number_of_taus']) * rabi['tau_step']+rabi['tau_start']
    if generate_new:
        sequencegenerator.delete_ensemble('Rabi')
        sequencegenerator.generate_rabi('Rabi', rabi['tau_start'], rabi['tau_step'], rabi['number_of_taus'], rabi['mw_freq'], 
                                        rabi['mw_amp'], 'a_ch1',setup['laser_length'] , 1.0, setup['aom_delay'], 
                                        setup['wait_length'], setup['seq_trig'], setup['gate'])
        if 'Rabi' not in sequencegenerator.saved_pulse_block_ensembles:
            sequencegenerator.log.error('Ensemble generation of Rabi in notebook timed out.')
            return
        pulsedmaster.sample('Rabi', True)
    else:
        pulsedmaster.load_asset_into_channels('Rabi')
    while pulsedmaster.status_dict['sauplo_ensemble_busy'] or pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.2)
    pulsedmaster.do_fit('No Fit')
    # set parameters in analysis tab
    pulsedmaster.measurement_sequence_settings_changed(tau_arr, rabi['number_of_taus'], 100e-6, [], False)
    pulsedmaster.fast_counter_settings_changed(setup['fc_binwidth'], setup['laser_length'])
    pulsedmaster.analysis_interval_changed(1)
    # perform measurement
    pulsedmaster.start_measurement()
    while not pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.2)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < rabi['measurement_time']:
        if not pulsedmaster.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmaster.manually_pull_data() 
    time.sleep(1)
    pulsedmaster.stop_measurement()
    while pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.2)
    if save_tag is None:
        pulsedmaster.save_measurement_data('s','Rabi_'+setup['NV_name']+'_'+str(rabi['mw_amp'])+'V',True)
    else:
        pulsedmaster.save_measurement_data('s',save_tag,True)
    time.sleep(2)
    # do fit
    x, y, param_dict = pulsedmeasurement.do_fit('Rabi')
    new_freq = param_dict.params['frequency'].value
    contrast = 2.*param_dict.params['amplitude'].value
    offset = param_dict.params['offset'].value
    period = 1./new_freq
    
    # write logfile
    write_to_logfile('Rabi_log', str(datetime.datetime.now()),name= setup['NV_name'],amplitude=rabi['mw_amp'], period=period,
                     frequency=new_freq, contrast=contrast, offset= offset)
    return user_terminated, period, contrast, new_freq, offset

def do_odmr(generate_new=True,save_data=False):
    # generate ensemble object
    if odmr['pi_length']>3e-3:
        print("Pulsed ODMR exceeds length of 3ms")
        return
        
    start_freq = odmr['resonance_freq']-(odmr['points']//2)*odmr['freq_res']
    freq_arr = start_freq + np.arange(odmr['points']) * odmr['freq_res']
    if generate_new:
        sequencegenerator.delete_ensemble('PulsedODMR')
        sequencegenerator.generate_pulsedodmr('PulsedODMR', odmr['pi_length'], start_freq, odmr['freq_res'], 
                                              odmr['points'], odmr['mw_amp'], 'a_ch1', setup['laser_length'] , 
                                              0.0, setup['aom_delay'], setup['wait_length'], setup['seq_trig'], setup['gate'])
        if 'PulsedODMR' not in sequencegenerator.saved_pulse_block_ensembles:
            sequencegenerator.log.error('Ensemble generation of PulsedODMR from notebook did not work.')
            return
        pulsedmaster.sample_block_ensemble('PulsedODMR', True)
    else:
        pulsedmaster.load_asset_into_channels('PulsedODMR')
    while pulsedmaster.status_dict['sauplo_ensemble_busy'] or pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.5)
    # delete old fit
    pulsedmaster.do_fit('No Fit')
    # set parameters in analysis tab
    pulsedmaster.measurement_sequence_settings_changed(freq_arr, odmr['points'], 100e-6, [], False)
    pulsedmaster.fast_counter_settings_changed(setup['fc_binwidth'], sequencegenerator.get_saved_asset("PulsedODMR").length_s)
    pulsedmaster.analysis_interval_changed(1)
    # perform measurement
    pulsedmaster.start_measurement()
    while not pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < odmr['measurement_time']:
        if not pulsedmaster.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(1)
    pulsedmaster.manually_pull_data() 
    time.sleep(1)
    pulsedmaster.stop_measurement()
    while pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
#    pulsedmaster.save_measurement_data('Hz','PulsedODMR'+setup['NV_name'])
    time.sleep(2)
    if odmr['fit_function']=='Lorentian':
        x, y, param_dict= pulsedmeasurement.do_fit('Lorentian')
        new_freq = param_dict.params['center'].value
        contrast = param_dict.params['contrast'].value
        linewidth = param_dict.params['fwhm'].value
    if odmr['fit_function']=='15N':
        x, y, param_dict = pulsedmeasurement.do_fit('15N')
        maximal_contrast = param_dict.params['l0_amplitude'].value
        new_freq = param_dict.params['l0_center'].value
        contrast = maximal_contrast
        linewidth = param_dict.params['l0_fwhm'].value
        
    freq_shift = new_freq - odmr['resonance_freq']
    if save_data:
        pulsedmaster.save_measurement_data('Hz','ODMR_'+setup['NV_name'],True)
    # write logfile
    write_to_logfile('PulsedODMR_log', str(datetime.datetime.now()),name= setup['NV_name'], frequency=new_freq,
                     contrast=contrast,linewidth=linewidth, shift=freq_shift)
    return user_terminated, new_freq, contrast, freq_shift, linewidth



def do_xy8(generate_new=True, save_tag='',load_tag=''):
    # sanity check for long sequences:
    if xy8['tau_start']+xy8['tau_incr']*xy8['points'] > 50e-3 :
        print("Waveform exceeds 3 ms too long for AWG7112")
        return True,0,0,0,0
    # generate ensemble object
    tau_arr = xy8['tau_start'] + np.arange(xy8['points']) * xy8['tau_incr']
    if generate_new:
        sequencegenerator.delete_ensemble('XY8')
        sequencegenerator.generate_xy8_tau('XY8', xy8['rabi_period'], xy8['mw_freq'], xy8['mw_amp'],
                                            xy8['tau_start'], xy8['tau_incr'], xy8['points'], 
                                            xy8['order'], 'a_ch1', setup['laser_length'], 0.0, 
                                            setup['aom_delay'], setup['wait_length'], setup['seq_trig'], 
                                            setup['gate'],xy8['alternating'])
        # sample, upload and load waveform
        if 'XY8' not in sequencegenerator.saved_pulse_block_ensembles:
            sequencegenerator.log.error('Ensemble generation of XY8 in notebook timed out.')
            return
        pulsedmaster.sample_block_ensemble('XY8', True)
        
    else:
        pulsedmaster.load_asset_into_channels('XY8')
    while pulsedmaster.status_dict['sauplo_ensemble_busy'] or pulsedmaster.status_dict['loading_busy']:
        time.sleep(0.5)
    pulsedmaster.do_fit('No Fit')
    # set parameters in analysis tab
    if xy8['alternating']:
        pulsedmaster.measurement_sequence_settings_changed(tau_arr, 2*xy8['points'], 100e-6, [],xy8['alternating'])
    else:
        pulsedmaster.measurement_sequence_settings_changed(tau_arr, xy8['points'], 100e-6, [],xy8['alternating'])
    pulsedmaster.fast_counter_settings_changed(setup['fc_binwidth'], sequencegenerator.get_saved_asset("XY8").length_s)
    pulsedmaster.analysis_interval_changed(1)
    pulsedmeasurement.measurement_tag = save_tag
    # perform measurement
    pulsedmaster.start_measurement(load_tag)
    while not pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < xy8['measurement_time']:
        if not pulsedmaster.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmaster.stop_measurement(save_tag)
    while pulsedmaster.status_dict['measurement_running']:
        time.sleep(0.5)
    if save_tag=='':
        pulsedmaster.save_measurement_data('s','XY8-'+str(xy8['order'])+'_'+setup['NV_name'],True)
    
    time.sleep(2)
    return user_terminated

def do_multi_measurement(poi_list,measurement_list):
    for poi in poi_list:
        if poi != 'crosshair' and poi !='sample':
            setup['NV_name']=poimanager.poi_list[poi]._name
            #checking stop condition
            user_terminated = False
            #Refocus current POI and calculate sample shift(all POI are shifted)    
            if 'odmr' in measurement_list:
                for i in range(3):
                    if refocus_poi(poi,min_counts=setup["min_counts"]):
                        result=do_odmr()
                        user_terminated=result[0]
                        rabi['mw_freq']=xy8['mw_freq']=result[1]
                        break
            if user_terminated:
                return True
            if 'rabi' in measurement_list:   
                for i in range(3):
                    if refocus_poi(poi,min_counts=setup["min_counts"]):
                        result=do_rabi()
                        user_terminated=result[0]
                        #set mw_amp
                        xy8['mw_amp']=rabi['mw_amp']
                        #set rabi_period                        
                        xy8['rabi_period']=result[1]  
                        break
            if user_terminated:
                return True
            if 'xy8' in measurement_list:   
                for i in range(3):
                    if refocus_poi(poi,min_counts=setup["min_counts"]):
                        if do_xy8_refocus(poi):
                            return True
                        break
            if user_terminated:
                return True                        
    return False

### refocus methods:

In [ ]:
def do_xy8_refocus(poi):
    end_measure=False
    #refocus_poi(poi)
    xy8_total_time = xy8['measurement_time']
    xy8_runtime = 0.0
    xy8['measurement_time'] = xy8['refocus_interval']
    end_measure = do_xy8(True, 'xy8_refocus','')
    xy8_runtime += xy8['refocus_interval']
    while xy8_total_time > xy8_runtime:
        for i in range(3):
            end_measure= not refocus_poi(poi)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure = do_xy8(False,'xy8_refocus', 'xy8_refocus')
        xy8_runtime += xy8['refocus_interval']
        if end_measure:
            break
    xy8['measurement_time'] = xy8_total_time
    # save measurement
    pulsedmaster.save_measurement_data('s','XY8-'+str(xy8['order'])+'_'+setup['NV_name'],True)
    write_to_logfile('xy8_log', str(datetime.datetime.now()),name= setup['NV_name'],runtime=xy8_runtime)
    time.sleep(2)
    return end_measure

# Laser On

In [ ]:
sequencegenerator.delete_ensemble('Laser_On')
sequencegenerator.generate_laser_on()
pulsedmaster.sample_block_ensemble('Laser_On', True)
while pulsedmaster.status_dict['sauplo_ensemble_busy'] or pulsedmaster.status_dict['loading_busy']:
    time.sleep(0.5)
pulsedmeasurement.pulse_generator_on()

# NV Parameters

In [ ]:
#NV20
standard_frequency = 1303.05e6
standard_drive_V = 0.25
standard_drive_Rabi = 21.31e6

# Rabi

In [ ]:
# for Rabi:
rabi = OrderedDict()
rabi['mw_amp'] = standard_drive_V
rabi['mw_freq'] =  standard_frequency
rabi['length'] = 200e-9
rabi['points'] = 50
rabi['measurement_time'] = 60


### There are several ways how to start the measurement

First one can just execute the rabi mehtod and do a rabi measurement 

In [ ]:
do_rabi()

One can also perform an experiment with multiple NVs and multiple measurements. Therefore the poi manager is used.
Here the poi is set to the active one in the poi manager.

In [ ]:
setup['poi'] = poimanager.active_poi.get_key()

here multiple measurements can be set, here only one poi is chosen in the list and also only one measurement: Rabi 

In [ ]:
do_multi_measurement([setup['poi']],['rabi'])

# ODMR

In [ ]:
# for pulsed ODMR:
odmr = OrderedDict()
factor = 10
odmr['mw_amp'] = standard_drive_V/factor
period = 1/standard_drive_Rabi
odmr['pi_length'] = period/2*factor
odmr['resonance_freq'] = standard_frequency
odmr['freq_res'] = 0.1e6
odmr['points'] = 100
odmr['measurement_time'] = 30
#odmr['fit_function']='15N'
odmr['fit_function']='Lorentian'
#odmr['fit_function']='14N'

There are also return values which can be read out

In [ ]:
end_measure, odmr_freq, odmr_contrast, odmr_shift, odmr_linewidth =  do_odmr()
print('ODMR frequency (MHz): ',odmr_freq/1e6)

In [ ]:
do_multi_measurement([setup['poi']],['odmr'])

# Example for multiple measurements with refocus

Here first an ODMR measurement is performed and based on the result a rabi measurement is done. The measured odmr frequency and rabi frequency are then used to perform the xy8 measuremnt. this measurement is stopped every 5min for a 
optical optimization. If one wants to use such a refocus it has to be implemented like in do_xy8_refocus(), in the same way a ODMR refocus can be done.

# XY8

In [ ]:
xy8 = OrderedDict()
xy8['mw_amp'] = standard_drive_V
xy8['mw_freq'] = standard_frequency
xy8['rabi_period']=1/standard_drive_Rabi
xy8['order'] = 8
frequency=2.203515e6
condition=1/2.
tau=condition/frequency
xy8['tau_incr'] =1e-9
xy8['points'] = 30
xy8['tau_start'] = tau-xy8['points']/2*xy8['tau_incr']
xy8['measurement_time'] = 30000
xy8['refocus_interval'] = 600
xy8['alternating']=True
print('tau center',tau)
print('start',xy8['tau_start'])
print('stop',xy8['tau_start']+xy8['points']*xy8['tau_incr'])
print('length start',xy8['tau_start']*8*xy8['order']*1e3)
print('length stop',(xy8['tau_start']+xy8['points']*xy8['tau_incr'])*8*xy8['order']*1e3)

In [ ]:
do_multi_measurement([setup['poi']],['odmr','rabi','xy8'])